In [21]:
# create model class
import torch
import torch.nn as nn

class Model(nn.Module): 
    # nn.Module is the base class from which we inherit all the offering

    def __init__(self, num_features):
        super().__init__()  # used in inheritance
        # self.linear1 = nn.Linear(num_features , 3) # input is num_features and output is 3
        # self.relu = nn.ReLU()
        # self.linear2 = nn.Linear(3,1)
        # self.sigmoid = nn.Sigmoid()
        
        self.network = nn.Sequential(
            nn.Linear(num_features,3),
            nn.ReLU(),
            nn.Linear(3,1),
            nn.Sigmoid()
        )
        
        
    def forward(self, features):
        out = self.network(features)
        
        return out
        

In [22]:
# create a dataset
features = torch.rand(10,5)

# create model
model = Model(features.shape[1]) # creating instance of model class

# call model for forward pass
model(features) # pytorch recommends not to use model.forward(features)

tensor([[0.5244],
        [0.5350],
        [0.5177],
        [0.5224],
        [0.5101],
        [0.5240],
        [0.5265],
        [0.4930],
        [0.5263],
        [0.5251]], grad_fn=<SigmoidBackward0>)

In [7]:
!pip install torchinfo

In [17]:
from torchinfo import summary

In [20]:
summary(model, input_size = (10,5))

Layer (type:depth-idx)                   Output Shape              Param #
Model                                    [10, 1]                   --
├─Linear: 1-1                            [10, 3]                   18
├─ReLU: 1-2                              [10, 3]                   --
├─Linear: 1-3                            [10, 1]                   4
├─Sigmoid: 1-4                           [10, 1]                   --
Total params: 22
Trainable params: 22
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00

## Applying On Previous Video Example

In [25]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [26]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [27]:
df.shape

(569, 33)

In [28]:
df.drop(columns = ['id', 'Unnamed: 32'], inplace = True)

In [29]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [30]:
X_train,X_test, y_train,y_test = train_test_split(df.iloc[:,1:], df.iloc[:,0], test_size = 0.2)

In [31]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [48]:
X_train

array([[-0.6994594 ,  0.18735489, -0.68971754, ..., -0.49529011,
         0.53295407,  0.34538881],
       [-0.25949431, -0.23708541, -0.30401036, ..., -0.13039476,
        -1.07063756, -0.76697815],
       [-0.17150129, -1.2180141 , -0.17473254, ...,  0.32376682,
         0.56763525,  0.55193201],
       ...,
       [-1.35471407, -0.21350539, -1.3441865 , ..., -0.97576174,
        -0.70896241, -0.10175524],
       [-0.84904753,  0.51511712, -0.87087659, ..., -1.0925909 ,
         0.15311259, -0.65821338],
       [-0.87251233, -0.09324731, -0.77094143, ...,  2.18582932,
         2.09856156,  3.13591924]])

In [33]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [34]:
X_train_tensor = torch.from_numpy(X_train)
X_test_tensor = torch.from_numpy(X_test)
y_train_tensor = torch.from_numpy(y_train)
y_test_tensor = torch.from_numpy(y_test)

In [52]:
X_train_tensor = X_train_tensor.float()

In [51]:
y_train_tensor = y_train_tensor.float()

### Defining the model

In [68]:
import torch.nn as nn

class MySimpleNN(nn.Module):
    def __init__(self,num_features):
        
        super().__init__()
        
        self.linear = nn.Linear(num_features,1)
        self.sigmoid = nn.Sigmoid() 
        
    def forward(self,features):
        
        out = self.linear(features)
        out = self.sigmoid(out)
        
        return out
    
        
    
    

### Important parameters


In [69]:
learning_rate = 0.1
epochs = 25

In [70]:
# define loss function
loss_function = nn.BCELoss()

### Training Pipeline


In [71]:
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr= learning_rate)

# define loop
for i in range(epochs):
    # foreward pass
    y_pred = model.forward(X_train_tensor)
    
    # loss calculate
    loss = loss_function(y_pred, y_train_tensor.reshape(-1,1))
    
    # it is recommended to cleaer gradient before backward pass
    optimizer.zero_grad()
    
    # backward pass
    loss.backward()
    
    # parameters update1
    optimizer.step()
    
    # print loss in each epoch
    print(f"Epoch : {i + 1} , Loss : {loss.item()}")
    
    

Epoch : 1 , Loss : 0.6697676181793213
Epoch : 2 , Loss : 0.5339567065238953
Epoch : 3 , Loss : 0.45540693402290344
Epoch : 4 , Loss : 0.4034615755081177
Epoch : 5 , Loss : 0.365972638130188
Epoch : 6 , Loss : 0.33731749653816223
Epoch : 7 , Loss : 0.3145178258419037
Epoch : 8 , Loss : 0.29583367705345154
Epoch : 9 , Loss : 0.2801724970340729
Epoch : 10 , Loss : 0.26680928468704224
Epoch : 11 , Loss : 0.255241334438324
Epoch : 12 , Loss : 0.24510760605335236
Epoch : 13 , Loss : 0.2361411303281784
Epoch : 14 , Loss : 0.2281394600868225
Epoch : 15 , Loss : 0.22094601392745972
Epoch : 16 , Loss : 0.214437335729599
Epoch : 17 , Loss : 0.20851467549800873
Epoch : 18 , Loss : 0.20309795439243317
Epoch : 19 , Loss : 0.19812147319316864
Epoch : 20 , Loss : 0.1935306042432785
Epoch : 21 , Loss : 0.18927977979183197
Epoch : 22 , Loss : 0.18533042073249817
Epoch : 23 , Loss : 0.18164972960948944
Epoch : 24 , Loss : 0.178209587931633
Epoch : 25 , Loss : 0.17498581111431122


In [72]:
model.linear.weight

Parameter containing:
tensor([[ 0.3606,  0.0462,  0.4338,  0.2647,  0.1450, -0.0421,  0.1557,  0.0935,
          0.1699, -0.0261,  0.2869, -0.0422,  0.3537,  0.1813, -0.0908,  0.1165,
         -0.1424,  0.1585,  0.0369, -0.1292,  0.0998,  0.3281,  0.1910,  0.1921,
          0.2221,  0.1603,  0.1113,  0.3079,  0.1250,  0.1690]],
       requires_grad=True)

In [73]:
model.linear.bias

Parameter containing:
tensor([-0.1013], requires_grad=True)

### Evaluation

In [74]:
# model evaluation
with torch.no_grad():
    y_pred = model.forward(X_test_tensor.float())
    y_pred = (y_pred > 0.5).float()
    accuracy = (y_pred == y_test_tensor).float().mean()
    print(f"Accuracy : {accuracy.item()}")    

    

Accuracy : 0.5012311339378357
